In [1]:
import numpy as np
import scipy as sp
import cvxpy as cp

In [2]:
# Reload random seed
np.random.seed(2025)
rng = np.random.default_rng()

# Parameters
NUM_DEPARTMENTS = 5
NUM_AUTHORS = 200
NUM_PRODUCTS = 1000

MULTIPLICATIVE_FACTOR = 2.5 ## The corretion term for the total number of paper submitted by each department
MAX_NUM_RP_PER_AUTHOR = 4
MIN_NUM_RP_PER_AUTHOR = 1
MAX_NUM_AUTHORS_RP_SAME_DEPARTMENT = 1
MAX_NUM_DEPARTMENTS_PRESENTING_SAME_RP = 2
MAX_GRADE = 5
MIN_GRADE = 1
AVG_GRADE = np.random.uniform(MIN_GRADE, MAX_GRADE, NUM_PRODUCTS)
VAR_GRADE = np.random.uniform(0, 1.5**2, NUM_PRODUCTS)

AUTHOR_WEIGHT = np.random.randint(1, 3, NUM_AUTHORS)
COVARIANCE_MATRIX = sp.sparse.bsr_array(np.diag(VAR_GRADE)) # sparse

In [3]:
## Make sure that all papers have (at least) one author and all authors have (at least) one paper
## and try to make it similar to a random assignment
AUTHORSHIP = cp.Variable((NUM_AUTHORS, NUM_PRODUCTS), boolean=True)
cp.Problem(
    cp.Minimize(cp.norm(AUTHORSHIP - np.random.randint(0, 2, (NUM_AUTHORS, NUM_PRODUCTS)), 1)), 
    [cp.sum(AUTHORSHIP, axis=0) >= 1, cp.sum(AUTHORSHIP, axis=1) >= 1]
).solve(verbose=True)

AUTHORSHIP = sp.sparse.bsr_array(AUTHORSHIP.value) # sparse
sp.sparse.save_npz("data/authorship.npz", AUTHORSHIP)

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Mar 25 10:37:22 PM: Your problem has 200000 variables, 2 constraints, and 0 parameters.
(CVXPY) Mar 25 10:37:22 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Mar 25 10:37:22 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 25 10:37:22 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Mar 25 10:37:22 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Mar 25 10:37:22 PM: Compiling problem (target solver=MOSEK).

In [4]:
AFFILIATION = rng.permuted(
    np.hstack([np.ones((NUM_AUTHORS,1)), np.zeros((NUM_AUTHORS,NUM_DEPARTMENTS-1))]),
    axis=1
)
AFFILIATION = sp.sparse.bsr_array(AFFILIATION) # sparse

# Variables
assignment = cp.Variable((NUM_AUTHORS, NUM_PRODUCTS), boolean=True)
is_rp_submitted = cp.Variable(NUM_PRODUCTS, boolean=True)

# Expressions
rp_per_author = cp.sum(assignment, axis=1)
num_assigned_rp_authors_per_department = assignment.T @ AFFILIATION

vqr_constraints = [
    # No authorship -> no assignment 
    assignment <= AUTHORSHIP,

    # Min and max number of RPs per author
    rp_per_author <= MAX_NUM_RP_PER_AUTHOR,
    rp_per_author >= MIN_NUM_RP_PER_AUTHOR,
 
    # Max number of assigned RP per author per department   
    num_assigned_rp_authors_per_department <= MAX_NUM_AUTHORS_RP_SAME_DEPARTMENT,
    
    # Definition of is_rp_submitted
    cp.sum(assignment, axis=0) >= is_rp_submitted,
    cp.sum(assignment, axis=0) <= cp.multiply(AUTHORSHIP.sum(axis=0), is_rp_submitted),
    
    # Max number of assigned RPs per department
    cp.sum(num_assigned_rp_authors_per_department, axis=0) <= np.floor(MAX_NUM_AUTHORS_RP_SAME_DEPARTMENT*AFFILIATION.sum(axis=0)),

    # RPs can be presented, at most, in two departments (following VQR4 guidelines)
    cp.sum(num_assigned_rp_authors_per_department, axis=1) <= MAX_NUM_DEPARTMENTS_PRESENTING_SAME_RP
]

# Objectives and indices
carrer_contribution = rp_per_author @ AUTHOR_WEIGHT
expected_evaluation = is_rp_submitted @ AVG_GRADE
# submission_risk = is_rp_submitted.T @ COVARIANCE_MATRIX @ is_rp_submitted
submission_risk = cp.abs(is_rp_submitted) @ VAR_GRADE

In [5]:
# Get the bounds for expected_evaluation 
max_expected_evaluation_prob = cp.Problem(cp.Maximize(expected_evaluation), vqr_constraints)\
                                 .solve(verbose=True)
max_expected_evaluation = expected_evaluation.value
min_expected_evaluation_prob = cp.Problem(cp.Minimize(expected_evaluation), vqr_constraints)\
                                 .solve(verbose=True)
min_expected_evaluation = expected_evaluation.value

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Mar 25 10:37:25 PM: Your problem has 201000 variables, 8 constraints, and 0 parameters.
(CVXPY) Mar 25 10:37:25 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Mar 25 10:37:25 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 25 10:37:25 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Mar 25 10:37:25 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Mar 25 10:37:25 PM: Compiling problem (target solver=MOSEK).

In [6]:
# Get the bounds for carrer_contribution
min_carrer_contribution_prob = cp.Problem(cp.Minimize(carrer_contribution), vqr_constraints)\
                                 .solve(verbose=True)
min_carrer_contribution = carrer_contribution.value
max_carrer_contribution_prob = cp.Problem(cp.Maximize(carrer_contribution), vqr_constraints)\
                                 .solve(verbose=True)
max_carrer_contribution = carrer_contribution.value

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Mar 25 10:38:36 PM: Your problem has 201000 variables, 8 constraints, and 0 parameters.
(CVXPY) Mar 25 10:38:36 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Mar 25 10:38:36 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 25 10:38:36 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Mar 25 10:38:36 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Mar 25 10:38:36 PM: Compiling problem (target solver=MOSEK).

In [ ]:
# Markowitz formulation
MARKOWITZ_TRADEOFF = 0
MAX_RISK = 15
MIN_EVALUATION = 0.4
normalized_evaluation_carrer_tradeoff = \
    (carrer_contribution - min_carrer_contribution)/np.maximum(1, max_carrer_contribution - min_carrer_contribution) + \
    (expected_evaluation - min_expected_evaluation)/np.maximum(1, max_expected_evaluation - min_expected_evaluation)

markowitz_tradeoff = \
    MARKOWITZ_TRADEOFF*normalized_evaluation_carrer_tradeoff - (1 - MARKOWITZ_TRADEOFF)*submission_risk

markowitz_constraints = []
if MARKOWITZ_TRADEOFF == 1:
    markowitz_constraints.append(
        # submission_risk <= COVARIANCE_MATRIX.sum() * (1 - np.floor(MARKOWITZ_TRADEOFF)) * MAX_RISK
        submission_risk <= MAX_RISK
    )
elif MARKOWITZ_TRADEOFF == 0:
    markowitz_constraints.append(
        # expected_evaluation >= np.ceil(MARKOWITZ_TRADEOFF) * MIN_EVALUATION
        normalized_evaluation_carrer_tradeoff >= MIN_EVALUATION
    )

markowitz = cp.Problem(cp.Maximize(markowitz_tradeoff), vqr_constraints + markowitz_constraints)
markowitz.solve(verbose=True)

                                     CVXPY                                     
                                     v1.4.2                                    
(CVXPY) Mar 25 10:38:48 PM: Your problem has 201000 variables, 9 constraints, and 0 parameters.
(CVXPY) Mar 25 10:38:48 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Mar 25 10:38:48 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Mar 25 10:38:48 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
(CVXPY) Mar 25 10:38:48 PM: Your problem is compiled with the CPP canonicalization backend.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Mar 25 10:38:48 PM: Compiling problem (target solver=MOSEK).